# Changes to Labor Market Racial Gap

Request BLS data on the unemployment rate and total employment for black, Hispanic, and white American, including youth (age 16-19). Examine the two year change in these figures. 

In [12]:
# Preliminaries (Python 2.7)
import requests
import json
import config # file called config.py with my API key
import pandas as pd

import math

%matplotlib inline

#### Set the date range 

Provide a start and end year to be used by the API.

In [13]:
# Include the start and end year here
date_range = (2015, 2017)

# Divide the date range into BLS-API-friendly length requests
req_no = int(math.ceil((date_range[1] - date_range[0]) / 10.0))
dates = []
for i in range(0,req_no):
    d1 = str(date_range[0]+i*10)
    d2 = str(date_range[0]+i*10+9)
    dates.append((d1,d2))
dates[-1] = (dates[-1][0], str(date_range[1]))

dates

[('2015', '2017')]

#### API and series information

Identify the series of interest and add headers to indicate json data should be read.

In [14]:
# URL, key, and headers same as above
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}

# This time, store the series into a dictionary with description
series_dict = {'LNS11000000': 'Overall Employment',
               'LNS14000000': 'Overall Employment Rate',
               'LNS11000006': 'Employment - Black',
               'LNS11000009': 'Employment - Hispanic',
               'LNS11000003': 'Employment - White',
               'LNS14000006': 'Unemp Rate - Black',
               'LNS14000009': 'Unemp Rate - Hispanic',
               'LNS14000003': 'Unemp Rate - White',
               'LNS11000018': 'Employment - Black - 16-19',
               'LNS11000021': 'Employment - Hispanic - 16-19',
               'LNS11000015': 'Employment - White - 16-19',
               'LNS14000018': 'Unemp Rate - Black - 16-19',
               'LNS14000021': 'Unemp Rate - Hispanic - 16-19',
               'LNS14000015': 'Unemp Rate - White - 16-19'}

df = pd.DataFrame()  # blank pandas dataframe to be filled later

#### Make requests for each date range in dates

In [15]:
for start, end in dates: 
    
    # The data sent in the post request now includes a start and end year
    data = json.dumps({"seriesid":series_dict.keys(), "startyear":start, "endyear":end})
    p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
    d = {} # New dictionary to be filled with data
    for series in p['Results']['series']:
        s = series['seriesID']  # Shorten name to 's'
        
        # Add dictionary entry with series and reverse series order
        d[s] = pd.DataFrame(series['data']).iloc[::-1]
        if len(series['data']) > 0:  # This if/else is to allow for series of different lengths
            
            # Convert BLS API dates to readable format (YYYY-MM-DD)
            d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])
            
            # Keep only date and series values
            d[s] = d[s].set_index('date')['value'].astype(float)
            
            # Rename and identify values as floating point numbers
            d[s] = d[s].rename(series_dict[s])
        else:  # If blank, leave as a blank pandas series
            d[s]['date'] = d[s]['value'] = pd.Series()
            d[s] = d[s]['value'].rename(series_dict[s])
            
    # Combine the dataframes for each range of years into one by appending        
    df = df.append(pd.concat([d[k] for k in series_dict.keys()], axis=1))

#### View the results

In [21]:
df.diff(24).tail(1)

,Unemp Rate - Hispanic,Employment - White - 16-19,Unemp Rate - White - 16-19,Unemp Rate - Black - 16-19,Unemp Rate - Black,Overall Employment Rate,Unemp Rate - White,Employment - Hispanic - 16-19,Overall Employment,Employment - White,Employment - Black,Employment - Hispanic,Employment - Black - 16-19,Unemp Rate - Hispanic - 16-19
date,,,,,,,,,,,,,,
2017-06-01,-1.9,146.0,-3.9,-10.2,-2.5,-0.9,-0.8,107.0,3152.0,1266.0,619.0,1133.0,49.0,-7.1


In [22]:
df.pct_change(24).tail(1)

,Unemp Rate - Hispanic,Employment - White - 16-19,Unemp Rate - White - 16-19,Unemp Rate - Black - 16-19,Unemp Rate - Black,Overall Employment Rate,Unemp Rate - White,Employment - Hispanic - 16-19,Overall Employment,Employment - White,Employment - Black,Employment - Hispanic,Employment - Black - 16-19,Unemp Rate - Hispanic - 16-19
date,,,,,,,,,,,,,,
2017-06-01,-0.283582,0.032416,-0.25,-0.325879,-0.260417,-0.169811,-0.173913,0.090909,0.020077,0.010241,0.031935,0.043315,0.067586,-0.342995
